In [1]:
import cv2
print( cv2.__version__ )
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

4.1.2


In [2]:
# Open a video file
# Note that OpenCV only extracts image frames but not audio 
vdo = cv2.VideoCapture("./Minion.mp4")

if vdo.isOpened():
    print('Video file opened successfully.')
else:
    print('Error. Video file could not be opened.')

Video file opened successfully.


In [3]:
bg = cv2.imread('BG.jpg')
bg.shape
# สูงคือ row = 752   กว้าง column = 1394

(752, 1394, 3)

In [4]:
while True:
    retval,img = vdo.read()
    if ( retval ):
        cv2.imshow('fgdg',img)
    if (cv2.waitKey(25)== 27):       # press ESC to quit
        break
        
print(img.shape)

(720, 1280, 3)


In [5]:
cv2.destroyAllWindows()             # close all windows
if vdo.isOpened(): 
    vdo.release()

In [16]:
# Open a video file
# Note that OpenCV only extracts image frames but not audio 
vdo = cv2.VideoCapture("./Minion.mp4")

if vdo.isOpened():
    print('Video file opened successfully.')
else:
    print('Error. Video file could not be opened.')

Video file opened successfully.


In [17]:
bg = cv2.imread('BG.jpg')

while True:
    retval,img = vdo.read()
    if ( retval ):
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        # แยก channel
        src_h = img_hsv[...,0]
        src_s = img_hsv[...,1]
        src_v = img_hsv[...,2]
        
        # threshold h
        thres1 = 50
        thres2 = 70
        # พื้นหลังอันนี้เป็นสีขาว มินเนี่ยนดำ
        mask_h = cv2.inRange(src_h, thres1, thres2)  #จากสีเขียว inrange เปลี่ยนเป็นสีขาว (bg)
        
        # ทำการสลับสีโดยใช้ not  ตอนนี้มินเนี่ยนเป็นสีขาวพื้นหลังดำ
        mask_h = cv2.bitwise_not(mask_h)
        
        # ใช้ thresh_binary คือ ถ้า ค่า pixel ไหนมากกว่า threshold เราจะกำหนดให้เป็น maxval ณ ที่นี่เราตั้ง = 255
        min_sat = 0.01
        ret, mask_s = cv2.threshold(src_s, min_sat*255, 255, cv2.THRESH_BINARY) #เกิน min_sat*255 ถึงให้เป็นขาว
    
        # รวม H S ตอนนี้เจอมินเนี่ยนเป็นสีขาว แต่พื้นหลังสีดำ
        mask_hs = cv2.bitwise_and(mask_h, mask_s)   # xor ตรงข้ามกับ and 
        
        # morp
        dilate = 3
        erode = 3
        ksize = 3
        kernel = cv2.getStructuringElement( cv2.MORPH_RECT, (ksize,ksize) )

        dilate = cv2.dilate(mask_hs, kernel, iterations = dilate)
        erode = cv2.erode(dilate, kernel, iterations = erode)
        img_morp = erode.copy()
        
        out = cv2.findContours(img_morp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(out) == 2:
            contours,hierarchy = out
        if len(out) == 3:
            _,contours,hierarchy = out
        
        draw_contour = np.zeros_like(img)
        cv2.drawContours(draw_contour, contours, -1, (255,255,255), thickness = -1) 
                                    # thickness = -1 คือ ระบายสีข้างในเส้น contour ทั้งหมด
        
        
        # result คือการจับภาพจริงมารวมกับที่ mask ไว้
        result = cv2.bitwise_and(img, draw_contour)  # ถ้าใช้ Morp ก็เปลี่ยน mask_hs เป็น img_morp แทน
        
        
        # resize  w ขึ้นก่อนเลยเอา index = 1
        # image shape ต้องปริ้นดูรอบนึง
        bg_re = cv2.resize(bg,(img.shape[1],img.shape[0]))
        
        # สร้าง condition เพื่อหาตำแหน่งที่จะสร้าง background สีดำ
        #condition = (result[...,0] ==0) & (result[...,1] == 0)& (result[...,2] == 0)
        condition = result[...] == 0
        result[condition] = bg_re[condition]       
        
        
#         cv2.inRange()
        cv2.imshow("Video", draw_contour)
    else:
        print("Error, no image from video file")

    # Wait 1 millisecond for any key press        
    if (cv2.waitKey(25)== 27):       # press ESC to quit
        break
vdo.release()
cv2.destroyAllWindows()  

In [ ]:
img.shape

In [19]:
# Open a video file
# Note that OpenCV only extracts image frames but not audio 
vdo = cv2.VideoCapture("./Minion.mp4")

if vdo.isOpened():
    print('Video file opened successfully.')
else:
    print('Error. Video file could not be opened.')

Video file opened successfully.


In [ ]:
# use edge detection
while True:
    retval,img = vdo.read()
    if ( retval ):
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        max_val = 10
        min_val = 2
        canny = cv2.Canny(img_gray,max_val, min_val)
        
        # morphological
        ksize = 5
        cnt_dilate = 5
        cnt_erode = 5
        
        kernel = cv2.getStructuringElement( cv2.MORPH_RECT, (ksize,ksize) )

        #morph_result = cv2.erode ( src_thresh, kernel, iterations=cnt_erode )
        morph_result = cv2.dilate ( canny, kernel, iterations=cnt_dilate )
        morph_result = cv2.erode ( morph_result, kernel, iterations=cnt_erode )
        
        # find contour line
        out = cv2.findContours(morph_result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(out) == 2:
            contours,hierarchy = out
        if len(out) == 3:
            _,contours,hierarchy = out
        
        draw_contour = np.zeros_like( img )
        cv2.drawContours ( draw_contour, contours, -1, (255,255,255), thickness = -1 )
        
        # แสดงผล
        result = cv2.bitwise_and(img, draw_contour)
  
        cv2.imshow("Video",result)
    else:
        print("Error, no image from video file")

    # Wait 1 millisecond for any key press        
    if (cv2.waitKey(50)== 27):       # press ESC to quit
        break
        
cv2.destroyAllWindows()             # close all windows
if vdo.isOpened(): 
    vdo.release()

In [4]:
cv2.destroyAllWindows()             # close all windows
if vdo.isOpened(): 
    vdo.release()